# IFT6168 coding assignment

In [1]:
import pandas
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import KFold
import numpy as np

In [2]:
df = pandas.read_csv('/cim/ehoney/ift6168_coding_assignment/simulated_data.csv')

In [3]:
df.head()

,Z1,Z2,W,T,M,Y
0,0.696714,0.064301,0.885308,0,1,0
1,0.061736,0.138900,0.144426,1,1,1
2,0.847689,0.080524,-0.286333,1,0,0
3,1.723030,0.222084,1.819394,1,0,1
4,-0.034153,0.439436,-2.136031,1,1,0


In [4]:
df.describe()

,Z1,Z2,W,T,M,Y
count,10000.000000,10000.000000,10000.000000,10000.0000,10000.000000,10000.00000
mean,0.197864,0.202707,-0.118175,0.5956,0.565600,0.48570
std,1.003462,0.200202,1.383609,0.4908,0.495703,0.49982
min,-3.722400,-0.571275,-6.187439,0.0000,0.000000,0.00000
25%,-0.472591,0.067598,-1.038364,0.0000,0.000000,0.00000
50%,0.197405,0.203169,-0.118197,1.0000,1.000000,0.00000
75%,0.871081,0.338773,0.825102,1.0000,1.000000,1.00000
max,4.126238,1.095817,4.877941,1.0000,1.000000,1.00000


## Reproducibility - setting random seed

Sample random entropy using NumPy SeedSequence

In [5]:
np.random.SeedSequence()

SeedSequence(
    entropy=100863271498176155769181426387931896632,
)

Truncate this to be small enough for a random seed

In [6]:
random_entropy = 142403440064326749963510733700818406553
random_seed = random_entropy // (10**29)
print(random_seed)

1424034400


Create KFold object for splitting dataset into folds for the experiments

In [7]:
num_folds=10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=random_seed)

Assign each row a unique fold index

In [8]:
for fold, (_, test_idx) in enumerate(kf.split(df)):
    df.loc[test_idx, 'fold'] = fold

Check this has been added correctly

In [9]:
df.head()

,Z1,Z2,W,T,M,Y,fold
0,0.696714,0.064301,0.885308,0,1,0,2.0
1,0.061736,0.138900,0.144426,1,1,1,3.0
2,0.847689,0.080524,-0.286333,1,0,0,8.0
3,1.723030,0.222084,1.819394,1,0,1,3.0
4,-0.034153,0.439436,-2.136031,1,1,0,6.0


## Question 2

Select our valid adjustment set

In [10]:
S = ["Z1", "Z2", "W"]

### 2(i) - Plug-in estimator with linear model of outcomes

We use linear regression to model $\mathbb{E}[Y|T=t,X=x]$ as $Q(t,x)$, where $X$ represents confounding variables in the valid adjustment set.

We just use the test set of each fold, since these are disjoint across the folds.

As suggested in the Murphy textbook, we can simply read off the regression coefficient for the treatment variable $T$ as our estimate of the ATE.

In [11]:
linear_plugin_estimates = []
for i in range(num_folds):
    split = df[df['fold']==i]

    model = LinearRegression()
    model.fit(split[S + ['T']], split['Y'])

    linear_plugin_estimates.append(model.coef_[-1])

In [12]:
linear_plugin_estimates = np.array(linear_plugin_estimates)

In [13]:
print(np.mean(linear_plugin_estimates))
print(np.std(linear_plugin_estimates))

0.0052859427167032125
0.022137884323740892


### 2(ii) - Plug-in estimator with nonlinear model of outcomes

Now we want to model $\mathbb{E}[Y|T=t,X=x]$ with a nonlinear function. We will use polynomial regression. We will perform a hyperparameter search on the maximum degree to use, selecting based on smallest mean squared error.

In [14]:
average_mses_for_degree = {}
for d in range(2, 8):
    poly = PolynomialFeatures(degree=d, include_bias=False)
    mse_vals = []
    for i in range(num_folds):
        train_split = df[df['fold']!=i]
        test_split = df[df['fold']==i]

        train_features = poly.fit_transform(train_split[S + ['T']])
        test_features = poly.fit_transform(test_split[S + ['T']])

        model = LinearRegression()
        model.fit(train_features, train_split['Y'])

        preds = model.predict(test_features)
        errors = test_split['T'] - preds
        mse = np.mean(errors**2)
        mse_vals.append(mse)
    average_mse = np.mean(np.array(mse_vals))
    average_mses_for_degree[d] = average_mse
    print(f"For degree = {d}, average MSE = {average_mse}")

For degree = 2, average MSE = 0.44572499691508394
For degree = 3, average MSE = 0.4554575323769052
For degree = 4, average MSE = 0.45617021749875064
For degree = 5, average MSE = 0.459837680968558
For degree = 6, average MSE = 0.4684469497760223
For degree = 7, average MSE = 0.46984934547618035


The mean squred error on test data appears to increase slightly with degree, so we will choose degree equal to 2 (a simple yet nonlinear model of outcomes).

Since our polynomial regression model includes interaction terms between the treatment variable $T$ and the confounding variables in our adjustment set $S$, we cannot simply read off the regression coefficient of $T$ as our effect estimate. Instead, we must use the general formula for the plug-in estimator.

In [15]:
poly = PolynomialFeatures(degree=2, include_bias=False)

nonlinear_plugin_estimates = []
for i in range(num_folds):
    split = df[df['fold']==i]

    features = poly.fit_transform(split[S + ['T']])

    model = LinearRegression()
    model.fit(features, split['Y'])

    test_input_with_treatment = split[S].copy()
    test_input_without_treatment = split[S].copy()

    test_input_with_treatment['T'] = 1
    test_input_without_treatment['T'] = 0

    test_input_with_treatment = poly.fit_transform(test_input_with_treatment)
    test_input_without_treatment = poly.fit_transform(test_input_without_treatment)

    nonlinear_plugin = np.mean((model.predict(test_input_with_treatment) - model.predict(test_input_without_treatment)))
    nonlinear_plugin_estimates.append(nonlinear_plugin)

In [16]:
nonlinear_plugin_estimates = np.array(nonlinear_plugin_estimates)

In [17]:
print(np.mean(nonlinear_plugin_estimates))
print(np.std(nonlinear_plugin_estimates))

0.00554072499982334
0.0234180759722199


### 2(iii) - Inverse propensity weighting estimator with linear model of treatments

Now we want to estimate the propensity score function $g(x) = \mathbb{P}(T=1|X=x)$ by fitting a logistic regression model, before computing the estimator using the IPTW estimator formula.

In [18]:
iptw_estimates = []
for i in range(num_folds):
    split = df[df['fold']==i]
    # split = df
    model = LogisticRegression()
    model.fit(split[S], split['T'])

    Y = split['Y']
    T = split['T']
    g_pred = model.predict_proba(split[S])[:,1] # indexing the T=1 probability

    iptw = np.mean(Y*T/g_pred - Y*(1-T)/(1-g_pred)) # applying iptw estimator formula

    iptw_estimates.append(iptw)

In [19]:
iptw_estimates = np.array(iptw_estimates)

In [20]:
print(np.mean(iptw_estimates))
print(np.std(iptw_estimates))

0.010142491037043486
0.031260712142922885


### Naive baseline that performs no adjustment

For this we will simply fit a linear regression model for predicting the outcome $Y$ from the treatment $T$.

In [21]:
naive_estimates = []
for i in range(num_folds):
    split = df[df['fold']==i]

    model = LinearRegression()
    model.fit(split[['T']], split['Y'])

    naive_estimates.append(model.coef_[-1])

In [22]:
naive_estimates = np.array(naive_estimates)

In [23]:
print(np.mean(naive_estimates))
print(np.std(naive_estimates))

-0.16194201170279193
0.02861518341087688


## Question 3

We will use the plug-in estimator with linear model of outcomes here. We have already evaluated it for one adjustment set so we will repeat for the remaining two.

In [24]:
S = ["Z1", "W"]

linear_plugin_estimates = []
for i in range(num_folds):
    split = df[df['fold']==i]

    model = LinearRegression()
    model.fit(split[S + ['T']], split['Y'])

    linear_plugin_estimates.append(model.coef_[-1])

linear_plugin_estimates = np.array(linear_plugin_estimates)

print(np.mean(linear_plugin_estimates))
print(np.std(linear_plugin_estimates))

0.0056102757211121375
0.022850143594064617


In [25]:
S = ["Z2", "W"]

linear_plugin_estimates = []
for i in range(num_folds):
    split = df[df['fold']==i]

    model = LinearRegression()
    model.fit(split[S + ['T']], split['Y'])

    linear_plugin_estimates.append(model.coef_[-1])

linear_plugin_estimates = np.array(linear_plugin_estimates)

print(np.mean(linear_plugin_estimates))
print(np.std(linear_plugin_estimates))

0.00454050657504
0.020404455594611882


## Question 4

We will include the mediator $M$ to make an invalid adjustment set. We will again use the linear plug-in estimator.

In [26]:
S = ["Z1", "Z2", "W", "M"]

linear_plugin_estimates = []
for i in range(num_folds):
    split = df[df['fold']==i]

    model = LinearRegression()
    model.fit(split[S + ['T']], split['Y'])

    linear_plugin_estimates.append(model.coef_[-1])

linear_plugin_estimates = np.array(linear_plugin_estimates)

print(np.mean(linear_plugin_estimates))
print(np.std(linear_plugin_estimates))

-0.0029568309728112557
0.023132405497617362
